In [72]:
from __future__ import annotations

from pathlib import Path
from typing import Optional, List, Dict, Any
import re

import pandas as pd
import plotly.graph_objects as go




### 配置

In [73]:
# -----------------------------
# 1) 配置：你的 per-image CSV 根目录
# -----------------------------
# 这里指向你导出 per-image CSV 的目录
# 例如：out_dir = Path("./_eval_exports_per_images") / version
PER_IMAGE_DIR = Path("./_eval_exports_per_images") / "sahi_null_v2"

# FiftyOne Web App 的 base（你之前是 https://fiftyone.tianqiyao.men）
FO_BASE = "https://fiftyone.tianqiyao.men"




### 工具

In [74]:
# -----------------------------
# 2) 工具：搜集 per-image CSV
# -----------------------------
def find_per_image_csvs(root: Path) -> List[Path]:
    # 你导出的命名：image_level__...csv
    return sorted(root.rglob("image_level_*.csv"))


# -----------------------------
# 3) 工具：构造 FiftyOne URL
# -----------------------------
def make_fo_url(dataset_name: str, sample_id: str, fo_base: str = FO_BASE) -> str:
    # 你之前用的格式：/datasets/{dataset_name}?id={sample_id}
    return f"{fo_base}/datasets/{dataset_name}?id={sample_id}"




### 核心：从 per-image CSV 画交互图，并导出 HTML（可点击）

In [75]:
# -----------------------------
# 4) 核心：从 per-image CSV 画交互图，并导出 HTML（可点击）
# -----------------------------
def draw_from_per_image_csv(
    csv_path: Path,
    output_dir: Optional[Path] = None,
    connectgaps: bool = True,
    show_plot: bool = True,
    y_cols: Optional[List[str]] = None,   # <-- 改：支持多个 y
    title: Optional[str] = None,
    add_cumulative: bool = False,         # <-- 可选：是否加 cumulative
):
    if y_cols is None:
        y_cols = ["tp_img"]  # 默认

    df = pd.read_csv(csv_path)

    required_cols = {"dataset_name", "sample_id", "filepath"}
    missing = required_cols - set(df.columns)
    if missing:
        print(f"[SKIP] missing columns {missing} in {csv_path}")
        return

    # 过滤掉不存在的 y_cols
    available_y = [c for c in y_cols if c in df.columns]
    missing_y = [c for c in y_cols if c not in df.columns]
    if missing_y:
        print(f"[WARN] missing y cols in {csv_path.name}: {missing_y}")
    if not available_y:
        print(f"[SKIP] no y cols available for {csv_path.name}")
        return

    # x 轴
    x_col = None
    if "capture_datetime" in df.columns:
        df["capture_datetime"] = pd.to_datetime(df["capture_datetime"], errors="coerce")
        if df["capture_datetime"].notna().any():
            x_col = "capture_datetime"
    if x_col is None:
        df["_index"] = range(len(df))
        x_col = "_index"
    df = df.sort_values(x_col)

    # URL
    df["sample_id"] = df["sample_id"].astype(str)
    df["url"] = df.apply(lambda r: make_fo_url(r["dataset_name"], r["sample_id"], FO_BASE), axis=1)

    # hover 信息
    hover_cols = []
    for c in ["filepath", "gt_count_img", "pred_count_img", "tp_img", "fp_img", "fn_img", "hit_img",
              "confidence_threshold", "iou_threshold", "model_tag"]:
        if c in df.columns:
            hover_cols.append(c)

    fig = go.Figure()

    # 多指标散点：每个指标一条 trace
    for y in available_y:
        yv = pd.to_numeric(df[y], errors="coerce").fillna(0)
        fig.add_trace(
            go.Scatter(
                x=df[x_col],
                y=yv,
                mode="lines+markers",
                name=f"{y} (per-image)",
                marker=dict(size=5),
                customdata=df[["url"] + hover_cols].values,
                hovertemplate=_build_hovertemplate(y_col=y, hover_cols=hover_cols),
                # connectgaps=connectgaps,
            )
        )

        # 可选 cumulative（每条 y 各自累计）
        if add_cumulative:
            fig.add_trace(
                go.Scatter(
                    x=df[x_col],
                    y=yv.cumsum(),
                    mode="lines",
                    name=f"{y} cumulative",
                    connectgaps=connectgaps,
                )
            )

    fig.update_layout(
        title=title or f"{csv_path.stem} | y={','.join(available_y)}",
        xaxis=dict(
            title="Time" if x_col == "capture_datetime" else "Index",
            rangeslider=dict(visible=True),
        ),
        template="plotly_white",
        margin=dict(l=70, r=70, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )

    if show_plot:
        fig.show()

    if output_dir is None:
        output_dir = csv_path.parent / "_plots"
    output_dir.mkdir(parents=True, exist_ok=True)

    out_html = output_dir / f"{csv_path.stem}__multi_y.html"

    post_script = """
    document.addEventListener("DOMContentLoaded", function() {
        var plot = document.getElementsByClassName('js-plotly-plot')[0];
        if (!plot) { return; }

        plot.on('plotly_click', function(data) {
            if (!data || !data.points || !data.points.length) { return; }
            var point = data.points[0];
            var url = point.customdata[0];
            if (url) window.open(url, '_blank');
        });
    });
    """

    fig.write_html(out_html, include_plotlyjs="cdn", full_html=True, post_script=post_script)
    print(f"[SAVE] {out_html}")


def _build_hovertemplate(y_col: str, hover_cols: List[str]) -> str:
    # customdata[0] 是 url，后面依次是 hover_cols
    lines = [f"<b>{y_col}</b>: %{{y}}<br>"]
    for i, c in enumerate(hover_cols, start=1):
        lines.append(f"{c}: %{{customdata[{i}]}}<br>")
    lines.append("Click point to open FiftyOne<br>")
    return "".join(lines)




### 批量跑：对所有 per-image CSV 生成图

In [76]:
# -----------------------------
# 5) 批量跑：对所有 per-image CSV 生成图
# -----------------------------
csvs = find_per_image_csvs(PER_IMAGE_DIR)
print("Found CSVs:", len(csvs))
csvs


Found CSVs: 126


[PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50.csv'),
 PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50.csv'),
 PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50.csv'),
 PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c80__iou50.csv'),
 PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c85__iou50.csv'),
 PosixPath('_eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c90__iou50.csv'),
 Pos

In [77]:
# 你可以在这里换你最想看的 y_col
Y_COL = "pred_count_img"  # 也可以试： "hit_img" / "fp_img" / "fn_img" / "pred_count_img"
Y_COL = "tp_img"  # 也可以试： "hit_img" / "fp_img" / "fn_img" / "pred_count_img"
Y_COLS = ["tp_img", "fp_img", "fn_img", "pred_count_img"]

for p in csvs[:10]:
    draw_from_per_image_csv(
        csv_path=p,
        connectgaps=True,
        show_plot=False,   # 批量时建议 False
        y_cols=Y_COLS,
        # add_cumulative=True,
    )

print("Done.")


[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c50__iou50__multi_y.html
[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c60__iou50__multi_y.html
[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c70__iou50__multi_y.html
[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c80__iou50__multi_y.html
[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1_ms2_0809__0823_10_ok_8__c85__iou50__multi_y.html
[SAVE] _eval_exports_per_images/sahi_null_v2/sahi_null_v2_ms1_0605__0621_40_ok/_plots/image_level_a03_yolo11n_custom7null_cv1